In [2]:
a = 'this is a book'
a.split()

['this', 'is', 'a', 'book']

In [5]:
import jieba
seg = jieba.cut('那酸民婉君也可以報名嗎?')
list(seg)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/jieba.cache
Loading model cost 0.781 seconds.
Prefix dict has been built succesfully.


['那', '酸民', '婉君', '也', '可以', '報名', '嗎', '?']

## Machine Learning

In [6]:

import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/pytextmining/master/data/20150628news.xlsx', index_col = 0)

In [7]:
news.head()

,category,description,pubdate,title
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中
2,財經/要聞,希臘債務違約限期逼近，資金持續外流。路透社引述三間銀行的消息指出，希臘國內有3分之1的自動櫃...,"Sun, 28 Jun 2015 07:40:00 +0800",希臘國內三分一自動櫃員機現金短缺
3,社會/生活,台鐵「新基隆車站」第一階段工程本月底完成，台鐵表示，明天（29號）啟用後，由於南站周邊道路尚...,"Sun, 28 Jun 2015 07:40:00 +0800",台鐵新基隆車站29日正式啟用
4,財經/要聞,《中國時報》\n●樂園變煉獄 派對驚爆 逾300人遭火紋身\n八仙樂園昨晚舉辦「COLOR ...,"Sun, 28 Jun 2015 07:38:17 +0800",6月28日各報頭版要聞


In [9]:
import jieba
#jieba.load_userdict('userdict.txt')

titles = []
corpus = []
for rec in news.iterrows():
    titles.append(rec[1].title)
    corpus.append(' '.join(jieba.cut(rec[1].description)))

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [12]:
X.shape

(147, 11138)

In [13]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
n_cosine_similarities  = cosine_similarity(X)

In [15]:
from sklearn import cluster
km = cluster.KMeans(n_clusters=4, init='k-means++', random_state=42)
c = km.fit_predict(n_cosine_similarities)

In [16]:
import numpy
np_titles = numpy.array(titles)

In [20]:
for t in np_titles[c == 3]:
    print(t)

希臘國內三分一自動櫃員機現金短缺
歐元區財長拒希臘延長救助計劃
呂紹煒專欄：違約與退出 希臘與歐洲才能重生(上)
希臘違約在即  歐盟全力穩定經濟
希臘脫歐變可能 歐洲衝擊大
希債協議  法國願盡最後斡旋努力
希臘1／3提款機錢被提光
確保銀行穩定 希臘續與ECB緊密合作
希臘態度強硬 歐元區耐心漸失
希臘盼展延債務 歐元區拒絕


In [22]:
import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/pytextmining/master/data/20171214news.xlsx')

In [23]:
news['category'].value_counts()

生活      160
社會      146
國際      127
財經地產    110
娛樂      101
政治       84
體育       62
論壇       38
副刊       25
3C       22
動物       19
壹週刊       5
Name: category, dtype: int64

In [24]:
corpus = []
titles = []
tags   = []
for rec in news[news['category'].isin(['社會','娛樂'])].iterrows():
    tags.append(rec[1]['category'])
    titles.append(rec[1]['title'])
    corpus.append(' '.join(jieba.cut(rec[1]['content'])))

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [26]:
X

<247x11380 sparse matrix of type '<class 'numpy.int64'>'
	with 30778 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.cross_validation import train_test_split
train_data, test_data, train_tag, test_tag, train_title, test_title = train_test_split(X, tags, titles, test_size=0.3, random_state=42)

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=0.01)
clf.fit(train_data,train_tag)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [29]:

predicted = clf.predict(test_data)

In [30]:
sum(test_tag == predicted) / len(test_tag)

0.9866666666666667

In [31]:
from sklearn.metrics import confusion_matrix
print(clf.classes_)
confusion_matrix(test_tag, predicted)

['娛樂' '社會']


array([[32,  0],
       [ 1, 42]])

In [33]:
import operator
coef_features_c1_c2 = []

for index, features in enumerate(zip(vectorizer.get_feature_names(), \
                        clf.feature_count_[0], clf.feature_count_[1])):
    feat,c1,c2 = features
    coef_features_c1_c2.append(tuple([c2/(c1 + 1), feat, c1, c2]))

for i in sorted(coef_features_c1_c2, key = operator.itemgetter(0), reverse=True)[0:20]:
    print(i)

(65.0, '男子', 0.0, 65.0)
(48.0, '警員', 0.0, 48.0)
(38.0, '突發', 0.0, 38.0)
(36.0, '新北', 0.0, 36.0)
(36.0, '機車', 0.0, 36.0)
(34.0, '江男', 0.0, 34.0)
(32.0, '進行', 0.0, 32.0)
(31.0, '法官', 0.0, 31.0)
(30.4, '警方', 4.0, 152.0)
(30.0, '少年', 0.0, 30.0)
(30.0, '張男', 0.0, 30.0)
(30.0, '派出所', 0.0, 30.0)
(29.0, '陳姓', 0.0, 29.0)
(29.0, '陳男', 0.0, 29.0)
(27.0, '林男', 0.0, 27.0)
(24.0, '時許', 0.0, 24.0)
(24.0, '檢方', 0.0, 24.0)
(24.0, '警察', 0.0, 24.0)
(23.5, '律師', 1.0, 47.0)
(23.0, '法院', 0.0, 23.0)


In [34]:
import pandas
movies  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/pytextmining/master/data/yahoo_movie.xlsx')

In [38]:
movies['title'].value_counts()

最終騎士       452
奧創紀元       281
攻殼機動隊      173
最後的絕地武士    117
古墓奇兵        89
Name: title, dtype: int64

In [39]:
movies = movies[movies['status'] != 'soso']

In [40]:
import jieba
#jieba.load_userdict('userdict.txt')
corpus = []
scores = []
for movie in movies.iterrows():
    corpus.append(' '.join(jieba.cut(movie[1].content)))
    scores.append(movie[1].status)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X

<980x4839 sparse matrix of type '<class 'numpy.float64'>'
	with 16211 stored elements in Compressed Sparse Row format>

In [42]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, scores, test_size = 0.3, random_state = 42)

In [43]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_X,train_Y)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [44]:
from sklearn.metrics import accuracy_score,confusion_matrix
predicted = clf.predict(test_X)

print(accuracy_score(test_Y, predicted))

0.7687074829931972


In [45]:
print(clf.classes_)
confusion_matrix(test_Y, predicted)

['bad' 'good']


array([[ 82,  35],
       [ 33, 144]])

In [47]:
import operator
coef_features_c1_c2 = []

for index, features in enumerate(zip(vectorizer.get_feature_names(), \
                        clf.feature_count_[0], clf.feature_count_[1])):
    feat,c1,c2 = features
    coef_features_c1_c2.append(tuple([c2/(c1 + 1), feat, c1, c2]))

for i in sorted(coef_features_c1_c2, key = operator.itemgetter(0), reverse=True)[0:20]:
    print(i)

(6.178434271448121, '不錯', 1.0217256810242579, 12.491099235107066)
(4.841496524639071, '好看', 5.721890998280063, 32.54401190717558)
(3.268002598004482, '非常', 0.8762828650075458, 6.131697277435952)
(3.113001433356311, '希望', 0.0, 3.113001433356311)
(3.0247294973540355, '雖然', 0.7251698131544237, 5.2181720217929355)
(2.611183199778206, '感動', 0.0, 2.611183199778206)
(2.571315326386349, '不過', 0.6261452957600632, 4.181332321918913)
(2.5367443211815166, '值得', 1.055760159296236, 5.214937909805936)
(2.472482266378667, '大家', 0.7358643195575264, 4.291893746945456)
(2.4505531251249275, '刺激', 0.4969358919588743, 3.6683209281514904)
(2.390246731027003, '其實', 0.627292613286546, 3.889630849332555)
(2.279531356498127, '爽片', 1.7571929836326827, 6.285107862107328)
(2.248390413545918, '有些', 0.33777782660848044, 3.0078468408008003)
(2.2296991856156567, '效果', 0.0, 2.2296991856156567)
(2.22153269208444, '但是', 0.7678723366638398, 3.927386191330429)
(2.2140501262756667, '喜歡', 1.0301665906033628, 4.494890596286016